# Making the final forward Index

In [19]:
import Data.saveFIndex.forwardindex_file_pb2 as fpb # Defining the proto buf data to store each element
from parse_forward import parse # Getting each document element from here
from wordlexicon import UpdateLexicon, lexicon_list # Updating the lexicon from here
import lexicon.lexicon_proto_file_pb2 as lexproto # Using this to update the lexicon as well
import os # using to get directory information of each json file
import Data.metadata_pb2 as mpb # Getting the metadata
import json
import time

In [20]:
from Data.doc_data import save_docs as save_doc # Helper function to store the doc data

In [21]:
import Data.doc_data.doc_file_pb2 as dpb # Storing the Doc ID with the data we will use to print later

In [22]:
%%time
metadata = mpb.MetaData()
metadatafile_path = "./Data/metadata.pb"
with open(metadatafile_path, "rb") as f:
    filecontent = f.read()
metadata.ParseFromString(filecontent)

forwardindex_limit = metadata.forwardindexlimit
total_docs = metadata.totaldocs
currdir = os.getcwd()
print(total_docs)
print(forwardindex_limit)

0
10000
CPU times: user 0 ns, sys: 1.2 ms, total: 1.2 ms
Wall time: 857 µs


#### Making the forward index

In [23]:
total_words = len(lexicon_list)
total_words

147408

In [24]:
%ls JSONFILES

all_jsons/  dir_1/  dir_2/  dir_3/  divfolders.ipynb


#### Defining the function:

In [25]:
os.listdir('JSONFILES/all_jsons')

['asiapacificresearch.json',
 'azmirror.json',
 'newyorkdailynews.json',
 'nationalobserver.json',
 'thedcclothesline.json',
 'unian.json',
 'curiousmindmagazine.json',
 'australiannationalreview.json',
 'theblaze.json',
 'themindunleashed.json',
 'nationalfile.json',
 'anonnews.json',
 'freedomainradio.json',
 'eluxemagazine.json',
 'thehuffingtonpostuk.json',
 'newsmax.json',
 'healthyfoodhouse.json',
 'southfront.json',
 'redstate.json',
 'spiegel.json',
 'motherjones.json',
 'naturallysavvy.json',
 'learntheriskorg.json',
 'thefiscaltimes.json',
 'hitandrun.json',
 'foreignpolicy.json',
 'healthsciencesinstitute.json',
 'dailysignal.json',
 'newsnetscotland.json',
 'slate.json',
 'vigilantcitizen.json',
 'thehill.json',
 'thoughtcrimeradio.json',
 'scientificamerican.json',
 'theguardian.json',
 'healingoracle.json',
 'thedailyexpress.json',
 'dailywire.json',
 'observer.json',
 'committeeconstructivetomorrowcfactorg.json',
 'thelibertydaily.json',
 'thepoke.json',
 'therightscoop.

In [26]:
with open("JSONFILES/all_jsons/newsmax.json", 'r') as f:
   json_data = json.load(f)
json_data[5]

{'id': "newsmax--2022-01-01--Rep. Murphy to Newsmax: AOC, Pelosi in 'Fantasy World' Over Tax Plans",
 'date': '2022-01-01',
 'source': 'newsmax',
 'title': "Rep. Murphy to Newsmax: AOC, Pelosi in 'Fantasy World' Over Tax Plans",
 'content': "House Speaker Nancy Pelosi and Rep. Alexandria Ocasio-Cortez are living in a `` fantasy world with unicorns and butterflies '' if they think the government can `` take other people 's money '' and pay for liberal agenda initiatives , Rep. Greg Murphy said on Newsmax Saturday .\n`` You know , I was sent something the other day from the IRS ruling that if you stole property , you were supposed to report that as income , '' the North Carolina Republican , who appeared on Newsmax 's `` Saturday Report '' with fellow Rep. Don Bacon , R-Neb.\n@ @ @ @ @ @ @ Democrats ' world is this possible . ''\nHe added that the Democrats ' calls for capital gains taxes , which are being rejected by Americans , is a `` total farce . ''\n`` This is grand larceny of the 

In [27]:
def addFIndex(dir_start, dir_end, no_of_barrels):
    doc_id = total_docs
    forwardindex = fpb.ForwardIndex()
    doc_data = dpb.DocData()
    count = 0
    start_time = time.time()
    for dir_index in range(dir_start, dir_end):
        dir_to_parse =  f"./JSONFILES/dir_{dir_index}"
        list_of_jsons = os.listdir(dir_to_parse)
        for each_json in list_of_jsons:
            json_path = os.path.join(dir_to_parse, each_json)
    
            with open(json_path, 'r') as f:
                data = json.load(f)
                for eacharticle in data:
                    forwardindex.docelement.append(parse(eacharticle, doc_id))
                    doc_data.eachdoc.append(save_doc.parse(eacharticle))
                    doc_id += 1
                    count += 1
                    if count % 500 == 0:
                        end_time = time.time()
                        print(count, "/", no_of_barrels*forwardindex_limit, " completed")
                        time_taken = end_time-start_time
                        print(f"Time taken: {time_taken}") 
                        start_time = time.time()
                        
                    if doc_id % forwardindex_limit == 0:
                        data_content = forwardindex.SerializeToString()
                        with open(f"./Data/forward_index/findex_{(doc_id//forwardindex_limit) - 1}.pb", "wb") as f:
                            f.write(data_content)
                        forwardindex = fpb.ForwardIndex()
                    if count / forwardindex_limit == no_of_barrels:
                        metadata.totaldocs = doc_id
                        print("Total final docs = ", doc_id)
                        with open(metadatafile_path, "wb") as f:
                            f.write(metadata.SerializeToString())
                        lexicon = lexproto.Lexicon()
                        lexicon.wordlist.extend(lexicon_list)
                        UpdateLexicon(lexicon)
                        return
    
    

In [28]:
addFIndex(1, 4, 11)

500 / 110000  completed
Time taken: 162.44662618637085
1000 / 110000  completed
Time taken: 152.0184452533722
1500 / 110000  completed
Time taken: 156.06553530693054
2000 / 110000  completed
Time taken: 166.54598546028137
2500 / 110000  completed
Time taken: 162.90471172332764
3000 / 110000  completed
Time taken: 157.73294925689697
3500 / 110000  completed
Time taken: 172.22956657409668
4000 / 110000  completed
Time taken: 162.1533694267273
4500 / 110000  completed
Time taken: 149.28699731826782
5000 / 110000  completed
Time taken: 142.3312270641327
5500 / 110000  completed
Time taken: 138.86605548858643
6000 / 110000  completed
Time taken: 141.30926132202148
6500 / 110000  completed
Time taken: 142.79642367362976
7000 / 110000  completed
Time taken: 142.50509023666382
7500 / 110000  completed
Time taken: 146.1170153617859
8000 / 110000  completed
Time taken: 143.81211304664612
8500 / 110000  completed
Time taken: 144.23888421058655
9000 / 110000  completed
Time taken: 141.229877471923

UnicodeDecodeError: 'utf-8' codec can't decode byte 0x91 in position 133029889: invalid start byte

In [13]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /home/arsal4an/nltk_data...


True

In [29]:
len(lexicon_list)

240956

In [31]:
%ls Data/forward_index

findex_0.pb  findex_2.pb  findex_4.pb  findex_6.pb
findex_1.pb  findex_3.pb  findex_5.pb  forwardindexdemo.pb


In [33]:
metadata.totaldocs = 70000
print("Total final docs = ", 70000)
with open(metadatafile_path, "wb") as f:
    f.write(metadata.SerializeToString())
lexicon = lexproto.Lexicon()
lexicon.wordlist.extend(lexicon_list)
UpdateLexicon(lexicon)

Total final docs =  70000
